## Import Statements

In [1]:
import os, csv, time, sqlite3
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from dbfuncs import *

## Defining Working Directories

In [9]:
working_dir = "/home/srivbane/shared/caringbridge/data/projects/qual-health-journeys/identify_candidate_sites"
valid_classification_sites_filename = os.path.join(working_dir, "valid_classification_sites.txt")
valid_sites_filtered_filename = os.path.join(working_dir, "valid_classification_sites_filtered.txt")

## Loading Site Data

In [10]:
if os.path.exists(valid_classification_sites_filename):
    with open(valid_classification_sites_filename, 'r') as infile:
        valid_sites = [int(line.strip()) for line in infile.readlines() if line.strip() != ""]
else:
    valid_sites = []
print(len(valid_sites))

58363


## Finding Frequency of words 'Hospice' & 'Funeral' in DataSet

In [12]:
def check_for_words(site_id):
    global counter, wordlist, total_size
    result_return = [0]*(len(wordlist)+1)
    result_return[0] = site_id
    
    if counter % 1000 == 0:
        print ((counter/total_size)*100)

    data = get_journal_text_all(site_id) 
    
    for journal in data:
        try:
            journal = journal.lower()
            for x in range(len(wordlist)):
                 result_return[x+1] += journal.count(wordlist[x])
        except :
            pass
    
    counter += 1
    return result_return


In [ ]:
counter = 1
wordlist = ["hospice","funeral","death","passed away","obituary","wake","commemoration"]
total_size = len(results)
pool = ThreadPool(24)
results = pool.map(check_for_words, valid_sites)
pool.close()
pool.join()

print (len (results))

In [14]:
csvfile = str(len(valid_sites))+"_results.csv"
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerow(["site_id"]+wordlist)
    for val in results:
        if sum(val[1:]) != 0:
            writer.writerow(val)